# <center id="c1"> 🛠 Tools и tools calling в LangChain 🦜🔗</center>



<img src='../images/agent_tool_calling.png' align="right" width="500" height="500" />

### Оглавление 

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🚀 Введение </a></li>
<li><a href="#c2">🛠 Инструменты для LLM 🤖</a></li>
<li><a href="#c3">Встроенные инструменты LangChain </a>
<li><a href="#c4">✍️🪡 Пишем инструменты сами!</a>
<li><a href="#c6">🗣 Tools calling = function calling</a>
<li><a href="#c7">🛠🪚 Пример с несколькими инструментами.</a>
<li><a href="#c7">Инструменты от OpenAI.</a>
<li><a href="#c5"> 🧸 Выводы и заключения ✅</a>
    
</ul></font></p>

### 🛠 В этом ноутбуке узнаем, как создавать инструменты и передавать их модели.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
📖 При всех выдающихся результатах `LLM` в решении различных задач, модель не сможет ответить на такой простой вопрос: "Какой сегодня день недели?" или о каком-то событии, которое произошло месяц назад, так не сохранила данную информацию в весах во время обучения. 

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
☝️  На помощь LLM приходят инструменты (`tools`) взаимодействия с внешним миром, которыми она может пользоваться при необходимости. 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Модель с инструментами, это основа для концепции  **автономных агентов**, кторую мы будем подробно рассматривать в следующем модуле.

In [1]:
from getpass import getpass
import warnings, os
warnings.filterwarnings('ignore')

In [2]:
!pip install langchain langchain_experimental langchain-classic langchain-community langchain-openai openai numexpr wikipedia -q -U

In [3]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [5]:
# Если используете ключ из курса, запустите эту ячейку
from langchain_openai import ChatOpenAI

#course_api_key= "Введите ваш ключ, полученный в боте курса"
course_api_key = getpass(prompt="Введите ваш ключ, полученный в боте курса")

# инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                 base_url="https://aleron-llm.neuraldeep.tech/")

Введите ваш ключ, полученный в боте курса ········


 #  <center id="c2"> 🛠🪚  Инструменты для LLM 🤖

📖 В `LangChain` существует несколько возможностей создавать и использовать инструменты:

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

* **Встроенные интеграции в фреймворк** (импортируются из `LangChain`):
   - `Tools` - просто импортируем и используем, например, Wikipedia, веб-поиск, Python REPL и пр.([полный список](https://python.langchain.com/docs/integrations/tools/))
   - `Toolkits` - целый ящик инструментов под задачу (например, `SQLToolkit`, `GitHubToolkit`)
* **Самописные инструменты** - их можно создавать различными способами из Python-функций или из сторонних API, на основе базовых классов.
* **Инструменты от вендоров моделей** - крупные вендоры (`OpenAI`, `Google`, `Anthropic`) предоставляют инструменты, которые можно использовать при обращении к моделям по API.

## <center>Встроенные инструменты `LangChain`</center>

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Начнём с инструментов, встроенных в `LangChain`, для примера возьмём инструмент `Wikipedia`, чтобы им воспользоваться необходимо дополнительно установить пакет для доступа к Википедии по АПИ: `!pip install wikipedia`

In [ ]:
#!pip install wikipedia -q

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Импортируем обёртку для доступа по API
api_wrapper = WikipediaAPIWrapper(top_k_results=1, # сколько результатов из поиска выводить
                                  doc_content_chars_max=134) # Максимальная длина саммари ответа

# Инициализируем инструмент
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

print(tool.invoke({"query": "LLM"}))

Page: Large language model
Summary: A large language model (LLM) is a language model trained with self-supervised machine learning on 


<div class="alert alert-success">

☝️ Как вы уже догадались, `tool` в `LangChain` тоже является `Runnable`, его можно также вызывать, использовать в цепочках и.т.п. 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Ещё один способ вызвать встроенный инструмент, это загрузить его по зарезервированному имени с помощью `load_tools`. <br>
Часто встречающиеся имена : `serpapi`, `llm-math`, `wikipedia`, `google-search`, `python_repl`, `requests`, `openweathermap`, `wolfram-alpha`, `arxiv` и др. Закрепленные имена смотрите в описании к каждому инструменту.

Для примера возьмём Википедию и калькулятор. 

In [7]:
from langchain.agents import load_tools

tools = load_tools(["wikipedia", "llm-math"], llm=llm)

# Получаем список из 2-х тулов
type(tools), len(tools)

(list, 2)

In [8]:
# Видим, что tool Wikipedia загрузился с параметрами по умолчанию
# Загружать топ-3 результата, на Английском, и 4000 символов
tools[0]

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/jovyan/venvs/sinara/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))

In [9]:
# Каждый инструмент должен иметь имя и описание, которые можно достать методами
tools[1].name, tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

In [10]:
# Проверим калькулятор
tools[1].invoke("Сколько будет (4.5*2.1)+2.2?")

'Answer: 11.650000000000002'

In [11]:
# Проверим
(4.5*2.1)+2.2

11.650000000000002

<div  class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
✅ Верно! Попробуем по сложнее:

In [12]:
tools[1].invoke('''У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.
            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?''')

'Answer: 23.0'

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Тоже верно! Но может получиться не с первого раза

### Так же с помощью `load_tools` можно подгрузить такой интересный tool:
```python
tools = load_tools(["human"])
```

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

📖 В документации к этому tool сказано:
> Human are AGI so they can certainly be used as a tool to help out AI agent when it is confused.

[Здесь](https://python.langchain.com/docs/integrations/tools/human_tools) можно прочитать про этот инструмент подробнее, пока ещё ИИ не может справиться без нас 🫣

## <center>✍️🪡 Пишем инструменты сами!</center>


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Тут важно заметить, что в `LangChain` есть 2 типа инструментов:
* класс `Tool` - принимает 1 аргумент на вход (функция с одним аргументом)
* класс `StructuredTool` - принимает на вход любое количество аргументов

☝️ Это стоит учитывать при написании собственных тулов.  

In [14]:
# Напишем функцию, которая считает количество символов в слове
def word_len(word):
    return len(word)
    
word_len('langchain')

9

In [15]:
from langchain_core.tools import Tool, StructuredTool

# создаём tool, так как 1 аргумент используем класс Tool
word_len_tool = Tool(
    name='Word Length',
    func=word_len,
    description='Считает количество символов в слове'
)

word_len_tool

Tool(name='Word Length', description='Считает количество символов в слове', func=<function word_len at 0x7f16bd8724d0>)

In [16]:
# Вызовем tool
word_len_tool.invoke('LLM')

3

In [17]:
word_len_tool.invoke('Автомобиль')

10

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Работает! Теперь попробуем с несколькими аргументами

In [23]:
# Добавим к функции несколько аргументов, и аннотации типов
def word_len_plus(word: str, b: int, g: int) -> int:
    """Считает количество символов в слове и прибавляет к ним числа""" # Обязательно добавляем docstring
    return len(word) + b - g

In [24]:
# другой способ создания тула методом from_function
word_len_tool_plus = StructuredTool.from_function(func=word_len_plus)

# Видим, что описание, название и пр. подхватились автоматически из docstring и аннотации
word_len_tool_plus

StructuredTool(name='word_len_plus', description='Считает количество символов в слове и прибавляет к ним числа', args_schema=<class 'langchain_core.utils.pydantic.word_len_plus'>, func=<function word_len_plus at 0x7f163c14b9a0>)

In [25]:
word_len_tool_plus.invoke({'word': 'LLM', 'b': 2, 'g': 3})

2

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
✅ **Отработал верно!** Если бы мы использовали просто `Tool` - упал бы с ошибкой.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

📖 Самый простой способ создавать `tools` и не думать сколько там аргументов у функции, это декоратор `@tool`, который автоматически определяет какой будет инструмент.

In [26]:
from langchain_core.tools import tool

@tool
def summ(a: int, b: int) -> int:
    """Складывает 2 целых числа"""
    return a + b

print(summ.name)
print(summ.description)
print(summ.args)

summ
Складывает 2 целых числа
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [27]:
summ # Декоратор сам определил, что это StructuredTool

StructuredTool(name='summ', description='Складывает 2 целых числа', args_schema=<class 'langchain_core.utils.pydantic.summ'>, func=<function summ at 0x7f163c14bb50>)

In [28]:
# попробуем сложить 2 числа 
summ.invoke({'a': 3, 'b': 5})

8

<div class="alert alert-warning">
    
<img src='../images/verr.png'>

Получим `ValidationError`, потому что на вход ожидаются целые числа.

## <center>🗣 Tools calling = LLM с инструментами </center>



Наделали мы кучу разных инструментов, как же их теперь использовать с нашей LLM?! 

<img src='../images/tc2.png'>

📖 **Смотрим на схему:**

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
1. **Создание инструмента** - с этим пунктом разобрались, инструмент импортировали или написали.
2. **Привязка инструмента** - инструмент должен быть подключен к модели, которая поддерживает вызов инструмента. Это дает модели информацию об инструменте и связанной схеме ввода, требуемой инструментом. Так же важно, чтобы ваша модель поддерживала `tool calling`
3. **Вызов инструмента** - при необходимости модель может принять решение о вызове инструмента и убедиться, что его ответ соответствует схеме ввода инструмента. Так же важо, чтобы ваша модель поддерживала `tool calling` (возможно вы слышали как `function calling`) - [список вендоров моделей](https://python.langchain.com/docs/integrations/chat/).
4. **Использование инструмента** - инструмент может быть запущен с использованием аргументов, предоставленных моделью.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Обобщив, можно сказать, что модель после получения пользовательского ввода смотрит какие у неё есть инструменты, решает какой(-ие) в данной ситуации применить (или никакого), вычленяет из запроса конкретные аргументы для подачи в инструмент.


Для примера возьмём наш инструмент `summ`, привяжем его к модели и сделаем запрос на естественном языке:

In [29]:
# Tool binding / привязка инструмента к LLM
model_with_tools = llm.bind_tools([summ])

# Tool calling 
response = model_with_tools.invoke("Сколько будет двадцать два плюс 17?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7ORv47m67OprAXSuXczm0MXX', 'function': {'arguments': '{"a":22,"b":17}', 'name': 'summ'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 59, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjEtuITwPJqaZAERVmWOay3DISd68', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--dc9177d8-1617-46bf-8d58-57b606a1d7a7-0', tool_calls=[{'name': 'summ', 'args': {'a': 22, 'b': 17}, 'id': 'call_7ORv47m67OprAXSuXczm0MXX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 59, 'output_tokens': 18, 'total_tokens': 77, 'input_token_details': {'audio': 0, 'cache_re

In [30]:
# Появился ключ tool_calls
response.tool_calls

[{'name': 'summ',
  'args': {'a': 22, 'b': 17},
  'id': 'call_7ORv47m67OprAXSuXczm0MXX',
  'type': 'tool_call'}]

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Модель верно решила использовать наш инструмент и достала правильные аргументы. Хотя написано "плюс" и числительные словами и цифрами.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**Как теперь использовать результат вызова?!** Передадим `tool_call` в инструмент и получим `ToolMessage` с результатом работы инструента:

In [31]:
response.tool_calls[0]

{'name': 'summ',
 'args': {'a': 22, 'b': 17},
 'id': 'call_7ORv47m67OprAXSuXczm0MXX',
 'type': 'tool_call'}

In [32]:
summ.invoke(response.tool_calls[0])

ToolMessage(content='39', name='summ', tool_call_id='call_7ORv47m67OprAXSuXczm0MXX')

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Возвращаем результат работы модели в инструмент и получаем правильный ответ.

Рассмотрим такой пример:

In [33]:
# Спросим нейтральный вопрос - модель выдала ответ, tool_calls не было.
model_with_tools.invoke('Какая столица Хорватии?')

AIMessage(content='Столицей Хорватии является Загреб.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 58, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjEvBseV0xBsQogr1ZkzfguJEMzoO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--3ff5bec8-4b72-409c-8c4f-7f3ecfe5a877-0', usage_metadata={'input_tokens': 58, 'output_tokens': 12, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [34]:
llm.invoke("Сколько букв в слове зачёт?").content

'В слове "зачёт" 6 букв.'

<div class="alert alert-warning" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**🤯 Ошибка!** Из-за особенностей токенизации, модель может галлюцинировать на такие простые вопросы.

Напишем `tool` ей в помощь:

In [35]:
@tool
def get_word_length(word: str) -> int:
    """Возвращает длину слова""" # добавляем docstring, чтобмы LLM понимала суть инструмента
    return len(word)

tools = [get_word_length]

In [37]:
# Tool binding
model_with_tools = llm.bind_tools(tools)

# Tool calling 
resp = model_with_tools.invoke("Сколько букв в слове зачёт?")
resp.tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'зачёт'},
  'id': 'call_D2zPUdWZ0RdHjJW7PAuM6GfY',
  'type': 'tool_call'}]

In [38]:
tool_message = get_word_length.invoke(resp.tool_calls[0])
tool_message

ToolMessage(content='5', name='get_word_length', tool_call_id='call_D2zPUdWZ0RdHjJW7PAuM6GfY')

In [39]:
from langchain_core.messages import HumanMessage

# теперь соберём все сообщения вместе 
# важно чтобы id в tool_calling и ToolMessage совпадали.
messages = [
    HumanMessage("Сколько букв в слове зачёт?"),
    resp,
    tool_message
]

model_with_tools.invoke(messages).content

'В слове "зачёт" 5 букв.'

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Теперь модель справилась с задачей!

# <center id="c4"> ✍️🤖 Пример с несколькими инструментами 🪭.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Возьмём `Wikipedia`, калькулятор, кастомный инструмент, и зададим модели каверзный вопрос.

In [40]:
import datetime

# Создадим инструмент, показывающий текущее время
@tool
def get_current_time(nothing: None ) -> str:
    """Возвращает текущее время""" 

    # Получаем текущее время и дату
    now = datetime.datetime.now()
    
    # Возвращаем текущее время
    return now.strftime('%H:%M:%S')

# Загрузим готовые инструменты
tools = load_tools(["wikipedia",
                    "llm-math"], llm=llm)

In [41]:
# Привязка инструментов к LLM
llm_with_tools = llm.bind_tools(tools + [get_current_time])

In [42]:
task = "Извлеки квадратный корень из девяти и назови количество участников группы Beatles. А еще скажи время"

messages = [HumanMessage(task)]

response = llm_with_tools.invoke(messages)
messages.append(response)
response.tool_calls

[{'name': 'Calculator',
  'args': {'__arg1': 'sqrt(9)'},
  'id': 'call_cPOURxHwSv4JmcKKEZmpAfuk',
  'type': 'tool_call'},
 {'name': 'wikipedia',
  'args': {'query': 'The Beatles'},
  'id': 'call_28d0Agl8REOi3OBK9rlL0OJY',
  'type': 'tool_call'},
 {'name': 'get_current_time',
  'args': {'nothing': None},
  'id': 'call_fp2xpPenmWZIhDrYeobfhD4L',
  'type': 'tool_call'}]

In [45]:
messages

[HumanMessage(content='Извлеки квадратный корень из девяти и назови количество участников группы Beatles. А еще скажи время', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cPOURxHwSv4JmcKKEZmpAfuk', 'function': {'arguments': '{"__arg1": "sqrt(9)"}', 'name': 'Calculator'}, 'type': 'function'}, {'id': 'call_28d0Agl8REOi3OBK9rlL0OJY', 'function': {'arguments': '{"query": "The Beatles"}', 'name': 'wikipedia'}, 'type': 'function'}, {'id': 'call_fp2xpPenmWZIhDrYeobfhD4L', 'function': {'arguments': '{"nothing": null}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 159, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Модель правильно определила аргументы для инструментов!

In [ ]:
# Соберём ToolMessages
for tool_call in response.tool_calls:

    if tool_call["name"] == 'wikipedia':
        tool_msg = tools[0].invoke(tool_call)
        
    elif tool_call["name"] == 'get_current_time':
        tool_msg = get_current_time.invoke(tool_call)
        
    else:
        tool_msg = tools[1].invoke(tool_call)
        
    messages.append(tool_msg)

In [47]:
print(llm_with_tools.invoke(messages).content)

Квадратный корень из девяти равен 3.0.

Количество участников группы Beatles — четыре: Джон Леннон, Пол Маккартни, Джордж Харрисон и Ринго Старр.

Текущее время: 01:04:03.


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Модель справилась, владея сразу тремя инструментами!

## <center> Инструменты от `OpenAI`.</center>


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

📖 Как правило, крупные вендоры языковых моделей предоставляют набор инструментов, доступных при обращении по `API`, у каждой компании это свой список. 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Инструметы от `OpenAI`:
* **Web search** - для поиска в интернете 
* **File search** - для поиска по файлу
* **Code interpreter** - для выполнений кода 
* **Remote MCP servers** - обращение к внешнем MCP серверку (аналог API)
* И некототорые другие ([список](https://platform.openai.com/docs/guides/tools?api-mode=responses) постоянно пополняется)

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

В данном случае указываем инструмент и сразу получаем ответ с его использованием.

<div class="alert alert-warning" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

⚠️ **ВАЖНО!** Каждый запрос с поиском тарифицируется по отдельной стоимости и может быстро исчерпать лимит на токены. (за 30 запросов можно исчерпать весь лимит на ключе курса по тарифу на 06.2025). Не рекомендуем запускать ячеки ниже без необходимости!

In [48]:
# Рассмотрим инструмент WebSearch
tool = {"type": "web_search_preview"}
llm_with_tools = llm.bind_tools([tool])

print(llm_with_tools.invoke("Какая температура в Сочи?").content[0]['text'])

В Сочи сейчас облачная погода с температурой около 15 °C.

## Погода для Сочи, Россия:
Текущие условия: Облачно, 59°F (15°C)

Ежедневный прогноз:
* вторник, июня 17: мин.: 58°F (15°C), макс.: 70°F (21°C), описание: Кратковременный ливень
* среда, июня 18: мин.: 64°F (18°C), макс.: 74°F (23°C), описание: Кратковременный ливень
* четверг, июня 19: мин.: 60°F (16°C), макс.: 75°F (24°C), описание: Кратковременный ливень
* пятница, июня 20: мин.: 63°F (17°C), макс.: 76°F (24°C), описание: Солнечно
* суббота, июня 21: мин.: 61°F (16°C), макс.: 73°F (23°C), описание: Небольшая облачность
* воскресенье, июня 22: мин.: 59°F (15°C), макс.: 74°F (24°C), описание: Солнечно
* понедельник, июня 23: мин.: 59°F (15°C), макс.: 73°F (23°C), описание: Преимущественно ясно


В течение дня ожидаются кратковременные ливни, с максимальной температурой около 21 °C и минимальной около 15 °C. 


<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Видим актуальную температуру на эту неделю!

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Так же возможен такой вызов инструмента:
```python
llm.invoke("...", tools=[{"type": "web_search_preview"}])
```

## <center> Так а где Агенты то?! 🕵‍♂️ </center>
### В следующем модуле!

Пример ниже работает только в версии `langchain < 1.0`

In [49]:
# import pandas as pd
# from langchain_classic.agents.agent_types import AgentType
# from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent


# url = "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
# df = pd.read_csv(url)

# agent = create_pandas_dataframe_agent(
#     llm,
#     df,
#     tools = [get_current_time],
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS,
#     allow_dangerous_code=True
# )

# agent.invoke("how many rows are there? And what is current time?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df)'}`


891
Invoking: `python_repl_ast` with `{'query': 'import datetime\ndatetime.datetime.now()'}`


2025-06-17 01:08:40.219960There are 891 rows in the dataframe. The current time is 2025-06-17 01:08:40.

> Finished chain.


{'input': 'how many rows are there? And what is current time?',
 'output': 'There are 891 rows in the dataframe. The current time is 2025-06-17 01:08:40.'}

<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
✅ Агент пояснил нам и за pandas и текущее время и даже не закашлял!

# <center id="c5"> 🧸 Выводы и заключения ✅ 

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Мы познакомились с одной из важных концепций в работе с LLM - `Tool Calling`, и посмотрели её реализацию в `LangChain`:

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
* **`Tool Calling`**: помогает вычленить из запроса на естественном языке необходимые аргументы и потом подать их в инструмент в нужном виде.
* Сразу будет проверено соответствие типов входных данных.
* Это позволяет получать более точные ответы от LLM в сложных случаях и уменьшить галлюцинации.
* Можно использовать сразу серию инструментов. Причем не писать инструменты с нуля, а брать готовые.
* Также `Function calling` является составной частью более сложной концепции **автономных агентов**, о которой мы расскажем в следующем модуле. 